In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
kakou = pd.read_csv('kakou.csv', header=None, names=['value'])

In [ ]:
first = kakou.ix[:1439]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(20, 10))

In [ ]:
plt.plot(first.ix[24*4*13:])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def create_dataset(dataset_raw, start, end, look_back=1, test=False):
    scaler = MinMaxScaler(feature_range=(0, 1))
    ds = np.ravel(scaler.fit_transform(np.array(dataset_raw[:].reshape(-1, 1))), 2)

    dataX, dataY, timeIndexY, forwards = [], [], [], []

    # 如果指定了起止时间

    start_index = start

    end_index = end

    a = ds[(start_index-look_back):start_index]
    
    dataX.append(a)
    if test:
#         dataY.append(ds[start_index:end_index+1, 0])
        dataY.append(ds[(start_index-look_back+1):end_index+1])
    else:
        dataY.append(ds[(start_index-look_back+1):start_index+1])
#         dataY.append(ds[start_index:start_index+1])

    # 不是构造测试集，并且目标值是填充值，跳过
    
    return np.array(dataX), np.array(dataY)

In [ ]:
cvX = []
cvY = []
for i in range(14, 15):
    for j in range(24, 80):
        x, y = create_dataset(first.value.values, 24*4*i+j, 24*4*i+j, 96)
        cvX.append(x)
        cvY.append(y)
cvX = np.vstack(cvX)
cvY = np.vstack(cvY)

In [ ]:
cvX.shape, cvY.shape

In [ ]:
import torch.utils.data as data
batch_size = 10
class MyDataset(data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):#返回的是tensor
        x, y = self.x[index], self.y[index]
        return x, y

    def __len__(self):
        return len(self.x)
    
cv_loader = torch.utils.data.DataLoader(
    MyDataset(cvX[:, :], cvY[:, :]), batch_size=batch_size, shuffle=False)

In [ ]:
trainX = []
trainY = []
for i in range(1, 13):
    for j in range(24, 80):
        x, y = create_dataset(first.value.values, 24*4*i+j, 24*4*i+j, 96)
        trainX.append(x)
        trainY.append(y)
trainX = np.vstack(trainX)
trainY = np.vstack(trainY)

In [ ]:
trainX.shape, trainY.shape

In [ ]:
train_loader = torch.utils.data.DataLoader(
    MyDataset(trainX[:, :], trainY[:, :]), batch_size=batch_size, shuffle=True)

In [ ]:
import pdb

In [ ]:
class Sequence(nn.Module):
    def __init__(self, input_dim):
        super(Sequence, self).__init__()
        self.hidden_dim = 51
        self.input_dim = input_dim
        self.lstm1 = nn.LSTMCell(input_dim, self.hidden_dim)
        self.lstm2 = nn.LSTMCell(self.hidden_dim, input_dim)
        self.hidden2out = nn.Linear(input_dim, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), self.hidden_dim).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), self.hidden_dim).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), self.input_dim).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), self.input_dim).double(), requires_grad=False)
#         pdb.set_trace()
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
#             pdb.set_trace()
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
#             out = self.hidden2out(c_t2)
            out = c_t2
            outputs += [out]
#             pdb.set_trace()
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(c_t2, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
#             out = self.hidden2out(c_t2)
            out = c_t2
            outputs += [out]
#         pdb.set_trace()
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [ ]:
seq = Sequence(1)
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.RMSprop(seq.parameters(), lr = 1e-3)
# optimizer = optim.LBFGS(seq.parameters())

In [ ]:
num_epochs = 5
losses = []
for epoch in range(num_epochs):
    print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
    for t, (x, y) in enumerate(train_loader):
        x_var = Variable(x).double()
#             x_var = Variable(torch.randn(30, batch_size, len(feature_selected)))
#             print x_var.type
        y_var = Variable(y).double()
#             pdb.set_trace()
        def closure():
            optimizer.zero_grad()
#             output = seq(x_var, y_var.size(1))
            output = seq(x_var)
#         loss = criterion(output[:, :2], y_var[:, :2])
#             weight = Variable(torch.log(torch.arange(output.size(1) + 1, 1, -1))).view(1, -1)
            weight = Variable(torch.ones(1, output.size(1)))
#             pdb.set_trace()
            mse = torch.pow(output - y_var, 2)
#             loss = torch.sum(mse * weight.double().expand_as(mse))
            loss = criterion(output, y_var)
            losses.append(loss.data.numpy()[0])
#             print('loss:', loss.data.numpy()[0])
            loss.backward()
#             pdb.set_trace()
#             print seq.lstm1.weight_hh.grad
            return loss
        optimizer.step(closure)
plt.plot(losses)

In [ ]:
result = []
for (cvx, cvy) in cv_loader:
        x_var = Variable(cvx).double()
        y_var = Variable(cvy).double()
        output = seq(x_var)
        result.append(output.data.numpy())

In [ ]:
np.vstack(result).shape, cvY.shape

In [ ]:
def mape(ypredict, ytrue):
    return np.mean(np.abs(ypredict - ytrue) / ytrue)

In [ ]:
mape(np.vstack(result)[:, -1], cvY[:, -1])

In [ ]:
plt.plot(np.ravel(cvY[:, -1]))
plt.plot(np.ravel(np.vstack(result)[:, -1]))